<a href="https://colab.research.google.com/github/rzens/myOriginalRzens/blob/master/Vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 连接到 google 云盘
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Dataset/
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Dataset
/content/drive/MyDrive/Dataset


In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import time

import torch
from torch.nn import Module, Sequential, Conv2d, ReLU, MaxPool2d, AdaptiveAvgPool2d, Dropout, Linear, CrossEntropyLoss, \
    Flatten
from torch.optim import Adam
from torch.utils.data import DataLoader

# from drive.MyDrive.Dataset import MyDataset

class Vgg16(Module):
    def __init__(self):
        super(Vgg16, self).__init__()
        self.features = Sequential(
            Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

            Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        self.avgpool = AdaptiveAvgPool2d(output_size=(7, 7))
        self.classifier = Sequential(
            Flatten(),
            Linear(in_features=25088, out_features=4096, bias=True),
            ReLU(inplace=True),
            Dropout(p=0.5, inplace=False),
            Linear(in_features=4096, out_features=4096, bias=True),
            ReLU(inplace=True),
            Dropout(p=0.5, inplace=False),
            Linear(in_features=4096, out_features=10, bias=True)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)

        x = self.classifier(x)
        return x


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_model_a_epoch(model: torch.nn.Module, loss, optim: torch.optim.Optimizer,
                        dataloader: torch.utils.data.dataloader, now_epoch: int):
    start_time = time.time()
    print("第 {} 次训练开始".format(now_epoch))
    model.train()
    for data in dataloader:
        imgs, targets = data
        # imgs=transform(imgs)
        imgs = imgs.to(device)
        targets = targets.to(device)
        train_output = model(imgs)
        train_loss = loss(train_output, targets)

        optim.zero_grad()
        train_loss.backward()
        optim.step()
    end_time = time.time()
    print("耗费时间 {}s".format(end_time - start_time))


def test_model_a_epoch(model: torch.nn.Module, loss,
                       dataloader: torch.utils.data.dataloader, now_epoch: int):
    start_time = time.time()
    print("第 {} 次测试开始".format(now_epoch))
    # 和性能记录相关的变量 #
    total_test_loss = 0
    total_test_accuracy = 0
    total_test_classes_accuracy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    total_test_classes_sum = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    # ############### #
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            imgs, targets = batch
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            test_loss = loss(outputs, targets)
            total_test_loss += test_loss.item()

            """
            output 是一个至少 shape 是 A = [ B1 = [], B2 = [], ...]
            .argmax(1) 将得到 B1 中最大值的那个 index
            """
            batch_accuracy = (outputs.argmax(1) == targets).sum()
            # 查看各类的准确度
            for classes0 in range(len(outputs.argmax(1))):
                total_test_classes_sum[targets[classes0]] += 1
                if outputs.argmax(1)[classes0] == targets[classes0]:
                    total_test_classes_accuracy[targets[classes0]] += 1

            total_test_accuracy += batch_accuracy.item()
    print("验证集损失：{}。".format(total_test_loss))
    print("本次 epoch 正确率：{}。".format(total_test_accuracy / test_data_size))
    for classes1 in range(len(total_test_classes_accuracy)):
        if total_test_classes_sum[classes1] == 0:
            continue
        print("本次 epoch 中第" + str(classes1) + "个类别的正确率：" +
              str(total_test_classes_accuracy[classes1] / total_test_classes_sum[classes1]) + "。",
              " total_test_classes_accuracy", total_test_classes_accuracy[classes1])
    end_time = time.time()
    print("耗费时间 {}s".format(end_time - start_time))

In [ ]:
if __name__ == "__main__":
    import torchvision
    train_data = torchvision.datasets.CIFAR10(root="drive/MyDrive/Dataset", train=True,
                                              transform=torchvision.transforms.ToTensor(),
                                              download=True)
    test_data = torchvision.datasets.CIFAR10(root="drive/MyDrive/Dataset", train=False,
                                            transform=torchvision.transforms.ToTensor(),
                                            download=True)
    test_data_size = len(test_data)
    train_dataloader = DataLoader(train_data, batch_size=256)
    test_dataloader = DataLoader(test_data, batch_size=256)

    vgg16 = Vgg16()

    device = torch.device("cpu")
    vgg16.to(device)
    ce_loss = CrossEntropyLoss()
    ce_loss.to(device)
    adam_optim = Adam(vgg16.parameters())

    # ##### intel ##### #
    # import intel_extension_for_pytorch as ipex

    # vgg16, adam_optim = ipex.optimize(vgg16, optimizer=adam_optim)
    # ##### ##### ##### #

    total_epochs = 30
    now_epoch = 0
    for i in range(total_epochs):
        train_model_a_epoch(vgg16, ce_loss, adam_optim, train_dataloader, now_epoch)
        test_model_a_epoch(vgg16, ce_loss, test_dataloader, now_epoch)
        now_epoch += 1

    torch.save(vgg16.state_dict(), "model.pth")  # 保存参数

Files already downloaded and verified
Files already downloaded and verified
第 0 次训练开始
